# Introduction to `pychegp`

*Author: Olivia Venot (CNRS/LISA/Univ. de Paris/Univ. Paris Est Créteil)*


`pychegp` is a 1D kinetic code that calculates the chemical composition of warm exoplanet atmospheres. Originally, `CHEGP` was a Fortran 90 code, but has been completety re-written in python by Ahmed Al-Refaie and plugged in with `TauREx 3.1`, which facilitates the calculation of observable outputs (e.g. synthetic spectra).

This tutorial will show you how to install and run the code `pychegp`, when used within the `TauREx 3.1` framework. It will show you how to modify the characteristics of the planet you want to study and how to plot the corresponding transmission and emission spectra.

Several options are possible to generate the planetary thermal profile. One of them is to use a thermal profile calculated by a radiative-transfer model.
In this tutorial, we will see how to use the code `exo_k`, written by Jérémy Leconte, to calculate thermal profiles and use them in `pychegp`.

## Installation

As the code will be used as a plugin in `TauREx 3.1`, you need to install this code by typing in your terminal
```
pip install taurex
```
You need to install the chemical equilibrium code `ACE` using the command line:
```
pip install taurex-ace
```
You need to install the code `pychegp` using the command line
```
pip install -i https://test.pypi.org/simple/ -U pychegp
```
Note that the code `pychegp` is not in a pubic version yet. By the way, any suggestion of a cool name for the code are very welcome before the code is publicly released !

## Initialization

To run the code, you need to import some common libraries:

In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook
from ipywidgets import *
import numpy as np
import sys
import taurex.log

import exo_k as xk
xk.Settings().set_mks(True)
import matplotlib.pyplot as plt
import matplotlib.colors as cols
import astropy.units as u
import time,sys
from exo_k.atm_evolution import Atm_evolution

TauREx logs can be disabled to hide intermediary messages:

In [2]:
taurex.log.disableLogging()

You can also create a nice environment to plot your figures with these lines:

In [3]:
plt.rcParams["figure.figsize"] = (7,4)
from matplotlib import cycler
colors = cycler('color',[plt.cm.inferno(i) for i in np.linspace(0.2,1,5)])
plt.rc('axes', axisbelow=True, grid=True, labelcolor='dimgray', labelweight='bold', prop_cycle=colors)
plt.rc('grid', linestyle='solid')
plt.rc('xtick', direction='in', color='dimgray')
plt.rc('ytick', direction='in', color='dimgray')
plt.rc('lines', linewidth=1.5)

To use `TauREx 3.1`, you need to have infrared cross-sections and collision-induced absorption data. Those are tables characterising the IR absorption of each species in the atmosphere. They depends on wavelength, temperature, and pressure. If you don't have them already on your computer, a directory with some sample files can be downloaded here:


*https://liveuclac-my.sharepoint.com/:f:/g/personal/ucapqch_ucl_ac_uk/Eupk2usw2e1ArmgluzbJ_XMBEO8q8daAZEhqdMxHxs817A?e=Fyk95l*.

In the following command line, simply replace the opacity and cia paths with the correct ones on your computer.

Before running `TauREx`, you have to load these IR data.

In [4]:
# load the cross section and cia information.

from taurex.cache import OpacityCache,CIACache
OpacityCache().clear_cache()
OpacityCache().set_opacity_path("/Users/olivia/Documents/TauREx/Inputs/xsec/taurex3_xsec_hdf5_sampled_R15000_0.3-50_2021MAY")
CIACache().set_cia_path("/Users/olivia/Documents/TauREx/Inputs/cia/hitran")


Similarly, for `exo_k`, you need to have IR cross section. `exo_k` uses data with a lower resolution. You will find these data here : *link*

## Definition of the planetary system

### The host star

You need to define the characteristics of the host star in the system you want to model. 

First, you define the flux of the star used by `TauREx` in order to calculate the synthetic spectra and eventually the thermal profile. 
You have two possibilities. First, you consider that the star has the irradiation of a black body. In this case, you import the library `Blackbodystar` and you need to indicate the `temperature`, the `radius` and the `mass` of the star. The command line is:

```
from taurex.stellar import BlackbodyStar
star = BlackbodyStar(temperature=3000,radius=1.0,mass=1.0)
```

Another possibility is to use a stellar model, like the Phoenix models (https://phoenix.ens-lyon.fr/simulator-jsf22-26/index.faces). In this case, you have to import the library `PhoenixStar`and indicate, in addition of the previous parameters, the path to the file to interpolate the stellar flux in `phoenix_path`.
The command is:
```
from taurex.stellar import PhoenixStar
star = PhoenixStar(temperature=3000,radius=1.0,mass=1.0, phoenix_path='folder')
```

As an example in this tutorial, we model the star as a black body of 6000 K:

In [ ]:
#STAR
from taurex.stellar import BlackbodyStar
# temperature in Kelvin
# radius in Sun radius
# mass in Sun mass
star = BlackbodyStar(temperature=6000,radius=1.203,mass=1.148)

The UV irradiation of the star, which is used by `pychegp` to calculate the photodissociations is defined later, when you set up the kinetic model.

*Note that this code can also be used to model atmospheres of Brown Dwarf. In this case, you don't need to define the host star.-> check with Quentin*

### The planet

To model the chemical composition of an exoplanet atmosphere, you need to set up the characteristics of your planet, by indicating the radius and the mass, in Jupiter units. You can also indicate the `name` of the planet. 
This parameter is usefull to store the output of the code.

```
from taurex.planet import Planet
planet = Planet(planet_radius=1.2,planet_mass=0.5)
name = 'PLANET'
```

As an example in this tutorial, we will study a hot Jupiter planet:

In [ ]:
#PLANET
from taurex.planet import Planet
# planet_radius in Jupiter radius
# planet_mass in Jupiter mass
planet_mass=0.69
planet_radius=1.34
planet = Planet(planet_radius=planet_radius,planet_mass=planet_mass)
name = 'example'

Now you can set up the thermal profile of the planet. You have several possibilities : isothermal profile, parametrized profile, or load an external profile calculated with the method of your choice:

#### Isothermal profile

To use an isothermal profile, you need to import the library `Isothermal` from `TauREx` and define the temperature (`T`).

```
from taurex.temperature import Isothermal
isothermal = Isothermal(T=1200)
```
#### Npoint profile

You can also use a parametrized profile calculated using the `Npoint` method of `TauREx`. In this case, you need to define the temperature (`T_surface`) and pressure (`P_surface`) at the surface, the ones at the top of the atmosphere (`T_top` and `P_top`), and the nodes between which the code interpolates in `pressure_points` and `temperature_points`. Temperatures are in Kelvin and pressures in Pascal.

```
from taurex.temperature import NPoint
NPoint = NPoint(T_surface=1200, T_top=800, P_surface=1e6, P_top=1e0, pressure_points=[1e3,], temperature_points=[1000,])
```

In [ ]:
from taurex.temperature import NPoint
NPoint = NPoint(T_surface=3000, T_top=1200, P_surface=1e7, P_top=1e-2, pressure_points=[1e5,1e3], temperature_points=[2000,1500])

#### Load an external profile
Finally, you can load the thermal profile of your choice, calculated with an external code for instance. You need to import the `TemperatureFile` library of `TauREx`. You need to indicate the correct path for your file, the columns corresponding to the temperature (`temp_col`) and the pressure (`press_col`), as well as the units (`temp_units` and `press_units`). The conversion to the units used by `TauREx`(SI) is done with the `astropy` library. You can indicate in `skiprows` the size of the header to be skipped if necessary.

```
from taurex.temperature import TemperatureFile
TemperatureFile = TemperatureFile(filename="path_to_your_file/profile.dat", skiprows=0, temp_col=2, press_col=1, temp_units='K', press_units='mbar',delimiter=None, reverse=False)
```


#### Compute a radiative convective equilibrium profile

To calculate the thermal profile of the planet, we will use the radiative-convective model `exo_k`.
First, you set up the data that you are using. You have to write the molecules you consider to calculate the opacity of the atmosphere in `molecules_w_kdata`. Indicate only species for which you have cross section data.
In `cia_molecules`, you indicate species for which you have CIA data.
You have to indicate where are these data in `search_path`.

In [ ]:
molecules_w_kdata=['H2O','CO','CH4','Na','PH3','TiO','VO','FeH','CO2','H2S','HCN','NH3']  # put here only the molecule for which we have cross sections
cia_molecules=['He', 'H2']  # put here the molecules for which there is only CIA

kdatabase=xk.Kdatabase(molecules_w_kdata,
            search_path='exo_k-public/R10_homogeneous_from_R500/',
            remove_zeros=True)
print(kdatabase)

cia_database=xk.CIAdatabase(molecules=cia_molecules)
cia_database.sample(wngrid=kdatabase.wns) # interpolates the database on the same spectral grid
print(cia_database)

if kdatabase.p_unit !='Pa':
    print("Careful, you are not using SI units... it will give you something, but things will go wrong eventually")

Then you define your atmosphere, with the number of layers (`Nlay`), the initial temperature at the surface (`T0`), and the temperature at the stratosphere (`Tstrat`). You also indicate the pressure at the surface (`psurf`) and at the top of the atmosphere (`ptop`).

You set the `internal_flux` of the planet, by indicating the internal temperature. This temperature is not well constrained and depends on the internal structure. It should be given by an interior model.
You can vary this parameter from 100 to $\sim$500 K typically. 
The `solar_flux` corresponds to the mean stellar irradiation (i.e. the flux at the substellar point, divided by 4). It has to be calculated taking into accoun the stellar luminosity and the semi-major axis.

You also define the composition of your atmosphere, by setting the volume mixing ratio of the main species in `bg_vmr`.

In [ ]:
Nlay=40; T0=2800.; Tstrat=1000.;
grav=xk.G*planet_mass*xk.MJUP/(planet_radius*xk.RJUP)**2;
psurf=1.e7; ptop=1.e0; 

internal_flux=xk.SIG_SB*500.**4
solar_flux=xk.SIG_SB*1200.**4

evol=Atm_evolution(Nlay=Nlay, Tsurf=T0, Tstrat=T0, psurf=psurf,
                   ptop=ptop, grav=grav,
                bg_vmr={'H2':'background', 'He':0.15,
                        'H2O':0.001, 'CO':0.001, 'Na':1.e-5},
                k_database=kdatabase,
                internal_flux=internal_flux,
                flux_top_dw=solar_flux,
                )
evol.evolve(Niter=1, N_kernel=10, verbose =False)   
trad=evol.tlay; trad2=trad; prad=evol.atm.play;

You can now run the `exo_k` model, to calculate the thermal profile of the planet.

In [ ]:
do_radiative_step=True
start=time.time()
time_init = evol.evol_time
if do_radiative_step:
    evol.set_options(dTmax_use_kernel=100., convection=False, radiative_acceleration=True)
    evol.evolve(Niter=500, N_kernel=1000, alpha=1., verbose=False)
    print('radiative equilibration step:',time.time()-start,'s')
    trad=evol.tlay; prad=evol.atm.play;
    evol.set_options(dTmax_use_kernel=100., convection=True, radiative_acceleration=True)
    evol.evolve(Niter=5, N_kernel=500000, alpha=1., verbose=False)
    trad2=evol.tlay;
evol.set_options(dTmax_use_kernel=100., convection=True, radiative_acceleration=False)
evol.evolve(Niter=200000, N_kernel=500000, alpha=1., verbose=False)
print('evol_time(yr):',(evol.evol_time-time_init)*evol.cp/(xk.DAY*365.))
print('timestep:',evol.timestep*evol.cp/(xk.DAY),'days, ',evol.timestep*evol.cp,'s')
print(evol.N_last_ker)
print('simulation time:',time.time()-start,'s')

You can plot the evolution with time of the fluxes (at the top of the atmosphere and internal).

In [ ]:
model_to_plot=evol
tot_mass=np.sum(model_to_plot.atm.dmass)
fig,axs=plt.subplots(1,2,sharey=False, figsize=(7,3))  
axs[0].plot(model_to_plot.Fnet_top, label='Top of Atmosphere')
axs[0].plot(np.ones_like(model_to_plot.Fnet_top)*internal_flux, label='internal flux')
axs[0].set_ylabel(r'Net Flux')
axs[0].set_xlabel(r'Timestep')
axs[0].legend()
fig.tight_layout()

And you can plot the heating in the atmosphere, the fluxes (radiative, convective and total), and the thermal profiles. The one that interest us the most in the "rad-conv equ".

In [ ]:
model_to_plot=evol
fig2,axs2=plt.subplots(1,3,sharey=True, figsize=(7,3))  
for label in ['rad','conv','tot']:
    axs2[0].plot(model_to_plot.H_ave[model_to_plot.header[label]]*xk.DAY/model_to_plot.cp, model_to_plot.atm.play, label=label)
    axs2[1].plot(model_to_plot.Fnet[model_to_plot.header[label]], model_to_plot.atm.play, label=label)
axs2[2].plot(trad, prad, label='radiative equ')
axs2[2].plot(trad2, prad, label='radiative equ2')
axs2[2].plot(model_to_plot.atm.tlay, model_to_plot.atm.play, label='rad-conv equ')
for ax in axs2:
    ax.legend(fontsize='xx-small', loc='upper right')
    ax.set_yscale('log')
axs2[0].set_ylabel(r'Pressure (Pa)')
axs2[0].set_xlabel(r'Heating (K/day)')
axs2[1].set_xlabel(r'Fluxes')
axs2[2].set_xlabel(r'T(K)')

axs2[0].invert_yaxis()
fig2.tight_layout()

In order to use this calculated thermal profile in the chemical model, to calculate the chemical composition, you have to save the data in an array. To do that, you import the `TemperatureArray` class of `TauREx` and type the following lines:

In [ ]:
from taurex.data.profiles.temperature.temparray import TemperatureArray
tp = TemperatureArray(tp_array=evol.atm.tlay[::-1], p_points=evol.atm.play[::-1], reverse=False)

## Elemental abundances

`ACEChemistry` and `pychegp` uses the elemental abundances defined in Lodders 2010, e.g. 
```
H = 12, He = 10.925, C = 8.39, N = 7.86, O = 8.73
```
These abundances are expressed in dex, with by definition `H = 12`.
The elemental abundance of `C` is not defined but is calculated depending on the C/O ratio required (`co_ratio`).
Setting the `co_ratio = 0.457` corresponds to solar elemental abundances.

# Calculation of the chemical composition and the observables

Now that you have set up the characteristics of the planetary system, you can calculate the chemical composition. First, we will explain how to calculate the thermochemical equilibrium, and in a second time, we will calculate the disequilibrium composition.

## Thermochemical equilibrium

You can calculate the chemical composition of the atmosphere at thermochemical equilibrium, using the library `ACEChemistry`. You just need to indicate the `metallicity` of the atmosphere (in solar units) and the C/O ratio, with the parameter `co_ratio`.

In [ ]:
# define equilibrium chemistry

from taurex_ace import ACEChemistry
chemistryACE = ACEChemistry(metallicity=1, co_ratio = 0.457)

### Radiative transfer model

#### Transmission model

Now that you have define the type of chemical composition you want to calculate, you can prepare the radiative transfer model you want to run. You can do both emission or transmission model. You need to indicate the `planet`, the `temperature_profile`, the `chemistry`, and the `star`, which have been defined previsouly. In addition, you have to indicate the pressure boundaries with `atm_min_pressure` and `atm_max_pressure`, as well as the number of layers of your model (`nlayers`).

In [ ]:
#transmission model with equilibrium chemistry
from taurex.model import TransmissionModel
tm_eq = TransmissionModel(planet=planet,
                       temperature_profile=tp,
                       chemistry=chemistryACE,
                       star=star,
                       atm_min_pressure=1e-2,
                       atm_max_pressure=1e7,
                       nlayers=100)

For the model to do something, we need to add some physics. E.g: contributions

In [ ]:
# add absorption contributions
from taurex.contributions import AbsorptionContribution
tm_eq.add_contribution(AbsorptionContribution())
# add cia contributions
from taurex.contributions import CIAContribution
tm_eq.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))
# add rayleigh contributions
from taurex.contributions import RayleighContribution
tm_eq.add_contribution(RayleighContribution())
# add clouds contribution. Clouds pressure is in Pa
from taurex.contributions import SimpleCloudsContribution
tm_eq.add_contribution(SimpleCloudsContribution(clouds_pressure=1e4))

The model needs to be built to initialize the different parts:

In [ ]:
tm_eq.build()

The model, using the thermochemical equilibrium composition, can now run. The results will be stored in `reseq`, but you can choose the name you want.

In [ ]:
res_eq = tm_eq.model()

#### Emission model

Similarly, you can prepare the emission model. You need to define your model, add contributions, build and run the model. For emission, we use the contribution `FlatMieContribution` for clouds. The parameters you can access are: the opacity value (`flat_mix_ratio`), the bottom of absorbing region in Pa (`flat_bottomP`), and the top of absorbing region in Pa (`flat_topP`)

In [ ]:
#emission model with equilibrium chemistry
from taurex.model import EmissionModel
em_eq = EmissionModel(planet=planet,
                       temperature_profile=tp,
                       chemistry=chemistryACE,
                       star=star,
                       atm_min_pressure=1e-2,
                       atm_max_pressure=1e7,
                       nlayers=100)

In [ ]:
# add absorption contributions
#from taurex.contributions import AbsorptionContribution
em_eq.add_contribution(AbsorptionContribution())

# add cia contributions
#from taurex.contributions import CIAContribution
em_eq.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))

# add rayleigh contributions
#from taurex.contributions import RayleighContribution
em_eq.add_contribution(RayleighContribution())

# add clouds contribution.
from taurex.contributions import FlatMieContribution
em_eq.add_contribution(FlatMieContribution(flat_mix_ratio=10,flat_bottomP=1e6, flat_topP=1e3))

In [ ]:
em_eq.build()

In [ ]:
res_em_eq = em_eq.model()

### Chemical composition

Once the code ran, you can see the chemical composition of the atmosphere. The best way to visualize it is to plot the abundances of the main species as a function of the pressure level. In the following example, we plot the active gases, as well as dihydrogene (H$_2$), atomic hydrogen (H), and Helium (He), which are inactive. In the following, we use the results of the transmission model, but you can use the ones of the Emission model as well, just replacing `tm_eq.` by `em_eq.`.

In [ ]:
#plot of thermochemical equilibrium

activechemistry_eq = tm_eq.chemistry.activeGasMixProfile
inactivechemistry_eq = tm_eq.chemistry.inactiveGasMixProfile
pressure_eq = tm_eq.pressure.profile*1e-2

colors=['blue','orange','green','cyan','navy','red','gold','olive','magenta','yellow','pink','grey']

fig, ax = plt.subplots()

## iterate over the active gases
for index, gas in enumerate(tm_eq.chemistry.activeGases):
    ax.plot(activechemistry_eq[index, :], pressure_eq,'-',color=colors[index], label=gas)
    
## iterate over the inactive gases. 
for index, gas in enumerate(tm_eq.chemistry.inactiveGases):
    if gas in ['H2', 'He', 'H']:
        ax.plot(inactivechemistry_eq[index, :], pressure_eq,'-', label=gas)    


ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e-12, 1)
ax.set_ylim(1e5, 1e-4)
ax.set_xlabel('Volume Mixing Ratio')
ax.set_ylabel('Pressure (mbar)')

plt.show()

You see that the atmosphere is dominated by H$_2$ and He, with important amount of CO and H$_2$O.

### Synthetic spectra

You can now plot the transmission and emission spectra corresponding to this chemical composition. 

#### Transmission Spectrum

You need first to unpack the results of your model (`res_eq`) in different variables. `res_eq` contains the grid of wavelengths (in fact wavenumbers), the transit depth, and the optical depth.

In [ ]:
# Transmission spectrum for equilibrim composition:
native_grid_eq, rprs_eq, tau_eq, _ = res_eq

`native_grid` is in wavenumber by convention. It is convenient to convert it in wavelengths if you like to work with this unit.

In [ ]:
native_grid_wl_eq = 10000/native_grid_eq

In [ ]:
plt.figure()
plt.plot(native_grid_wl_eq, rprs_eq, color='red')
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Transit depth (%)')
plt.xlim(0.5, 8)
plt.show()

This plot is at quite high resolution, and it's hard to see anything. You can bin the spectra using the util function `create_grid_res` and the binner `FluxBinner` from `TauREx`. Let's select only the range 0.5 to 8 $\mu$m and choose a resolution of 80.

In [ ]:
from taurex.binning import FluxBinner
from taurex.util.util import create_grid_res
# We choose the resolution and the range of wavelength
lowres_grid = create_grid_res(80, 0.5, 8.0)
# We initialise the binner with this grid, passing the wavelength and the bin size.
fb = FluxBinner(lowres_grid[:,0], lowres_grid[:,1])
# We can now bin the native spectrum (Note that the grid must be sorted by increasing wavelengths)
lowres_spectrum_eq = fb.bindown(native_grid_wl_eq[::-1], rprs_eq[::-1])

You can now plot the high and low resolution spectra in order to check that everything has been done right.

In [ ]:
plt.figure()
plt.plot(native_grid_wl_eq, rprs_eq, color='blue', label='High resolution spectrum')
plt.plot(lowres_spectrum_eq[0], lowres_spectrum_eq[1], color='orange', label='Low resolution spectrum')
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Transit depth (%)')
plt.legend()
plt.xlim(0.5,8.0)
plt.show()

#### Emission spectrum

To plot the emission spectrum, steps are the same:

In [ ]:
# Emission spectrum for equilibrim composition:
native_grid_em_eq, fpfs_em_eq, tau_em_eq, _ = res_em_eq

In [ ]:
native_grid_wl_em_eq = 10000/native_grid_em_eq

In [ ]:
plt.figure()
plt.plot(native_grid_wl_em_eq, fpfs_em_eq, color='red')
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('F$_p$/F$_s$')
plt.xlim(0.5, 8)
plt.ylim(0,0.001)
plt.show()

As for transmission spectrum, you can binned down the resolution:

In [ ]:
#from taurex.binning import FluxBinner
#from taurex.util.util import create_grid_res
# We choose the resolution and the range of wavelength
lowres_grid = create_grid_res(80, 0.5, 8.0)
# We initialise the binner with this grid, passing the wavelength and the bin size.
fb = FluxBinner(lowres_grid[:,0], lowres_grid[:,1])
# We can now bin the native spectrum (Note that the grid must be sorted by increasing wavelengths)
lowres_spectrum_em_eq = fb.bindown(native_grid_wl_em_eq[::-1], fpfs_em_eq[::-1])

In [ ]:
plt.figure()
plt.plot(native_grid_wl_eq, fpfs_em_eq, color='blue', label='High resolution spectrum')
plt.plot(lowres_spectrum_em_eq[0], lowres_spectrum_em_eq[1], color='orange', label='Low resolution spectrum')
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('F$_p$/F$_s$')
plt.legend()
plt.xlim(0.5,8.0)
plt.ylim(0,0.001)
plt.show()

## Disequilibrium composition

Now that you ran the model assuming thermochemical equilibrium, you can run the model taking into account the disequilibrium chemistry. This time you use the code `pychegp`, that is used as a library:`PychegpDiseq`. As for the calculation of chemical equilibrium, you need to define the `metallicity`, the `co_ratio`, but also parameters specific to the kinetic model, such as the eddy diffusion coefficient (`Kzz_value` expressed in cm$^2$.s${^-1}$). Another usefull parameter is `o_seq`, which permits to indicate the amount of oxygen you want to remove from the elemental abundance defined with `metallicity` and `co_ratio`. This permits to simulate a depletion for oxygen, due to the sequestration within silicates and metals.

Other arguments are :

`maximum_runtime` (in minutes) permits to stop the code if it has not converged yet.
`verbose`=True *DON'T KNOW*
`df_stop` and `dfdt_stop` are two criteria of convergence that need to be both fullfiled. They indicate that the atmosphere reached the steady state composition. `df` is the maximum variation of mixing ratio from an iteration to another and `dfdt` is the variation of the derivative with time. The default values are respectivly `0.0005` and `1e-4`.

You can also indicate the maximum simulation time (`t_final` in seconds) you want to reach, if the criteria of steady state have not been reached before.

If you want to take into account the influence of the star (and thus photodissociations), you need to indicate the flux of the star in `star_path`. The format of the file is wavelenght (in nm) in the first column, and  stellar flux in cm$^{-2}$.s$^{-1}$. If you don't indicate the flux, no photodissociation will be calculated.

`rejection_scheme` is for ...


`atol` and `rtol` are parameter for the solver. They correspond respectively to the absolute and relative tolerance. By default, the values are `atol=1e-25` and `rtol=1e-3`.

You can disable molecular diffusion with the option `disable_diffusion`. `False` means that diffusion is active.

Finally, there are optional entries, linked to the chemical scheme that you want to use. By default, the code will use the chemical scheme from Venot et al . 2020 (https://doi.org/10.1051/0004-6361/201936697). But you can indicate the chemical scheme of your choice, indicating the list of species in `spec_file`, and the folder with the files containing the chemical reactions in `data_path`. The thermodynamic data are the ones used by Venot et al. 2020, but you can use the file of your choice, indicated in `therm_file`.

In [ ]:
from pychegp.taurex import PychegpDiseq
chemistry = PychegpDiseq(metallicity = 1.0, co_ratio = 0.457, Kzz_value = 1e9,
                         o_seq=0.0, maximum_runtime=15.0, verbose=True , df_stop=0.0005, dfdt_stop=1e-4, 
                         t_final=1e12, star_path="pychegp/examples/stellarflux_Sun.dat", rejection_scheme='exception', 
                         atol=1e-25, rtol=1e-3, disable_diffusion=False)

### Radiative transfer model

Now that the chemistry has been set up as 'disequilibrium', you can prepare the calculation of the radiative transfer model, either transmission or emission, like you did for the thermochemical equilibrium model. Let's begin with transmission.

#### Transmission model

In [ ]:
#from taurex.model import TransmissionModel
tm_diseq = TransmissionModel(planet=planet,
                       temperature_profile=tp,
                       chemistry=chemistry,
                       star=star,
                       atm_min_pressure=1e-2,
                       atm_max_pressure=1e7,
                       nlayers=100)

Then, you need to add the contributions to the Transmission model:

In [ ]:
##add absorption contributions
#from taurex.contributions import AbsorptionContribution
tm_diseq.add_contribution(AbsorptionContribution())

##add cia contributions
#from taurex.contributions import CIAContribution
tm_diseq.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))

##add rayleigh contributions
#from taurex.contributions import RayleighContribution
tm_diseq.add_contribution(RayleighContribution())

##add clouds contribution. Clouds pressure is in Pa
#from taurex.contributions import SimpleCloudsContribution
tm_diseq.add_contribution(SimpleCloudsContribution(clouds_pressure=1e4))

and then build the model. It will be longer than with equilibrium chemistry, because the calculation of kinetics takes more time.

In [ ]:
tm_diseq.build()

After, the model with chemical kinetics can run:

In [ ]:
res_tm_diseq = tm_diseq.model()

#### Emission model

Let's do now the emission model for the disequilibrium composition

In [ ]:
#from taurex.model import EmissionModel
em_diseq = EmissionModel(planet=planet,
                   temperature_profile=tp,
                   chemistry=chemistry,
                   star=star,
                   atm_min_pressure=1e-2,
                   atm_max_pressure=1e7,
                   nlayers=100)

In [ ]:
##add absorption contributions
#from taurex.contributions import AbsorptionContribution
em_diseq.add_contribution(AbsorptionContribution())

##add cia contributions
#from taurex.contributions import CIAContribution
em_diseq.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))

##add rayleigh contributions
#from taurex.contributions import RayleighContribution
em_diseq.add_contribution(RayleighContribution())

##add clouds contribution. Clouds pressure is in Pa
from taurex.contributions import FlatMieContribution
em_eq.add_contribution(FlatMieContribution(flat_mix_ratio=10,flat_bottomP=1e6, flat_topP=1e3))

In [ ]:
em_diseq.build()

In [ ]:
res_em_diseq = em_diseq.model()

### Chemical composition

You can now look at the results of the model, especially the chemical composition.

In [ ]:
#plot of disequilibrium composition
activechemistry = tm_diseq.chemistry.activeGasMixProfile
inactivechemistry = tm_diseq.chemistry.inactiveGasMixProfile
pressure = tm_diseq.pressure.profile*1e-2

colors=['blue','orange','green','cyan','navy','red','gold','olive','magenta','yellow','pink','grey']

fig, ax = plt.subplots()

## iterate over the active gases
for index, gas in enumerate(tm_diseq.chemistry.activeGases):
    ax.plot(activechemistry[index, :], pressure,'-', color=colors[index], label = gas)
    
## iterate over the inactive gases. 
for index, gas in enumerate(tm_diseq.chemistry.inactiveGases):
    if gas in ['H2', 'He', 'H']:
        ax.plot(inactivechemistry[index, :], pressure,'-', label = gas)

ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e-12, 1)
ax.set_ylim(1e5, 1e-4)
ax.set_xlabel('Volume Mixing Ratio')
ax.set_ylabel('Pressure (mbar)')

#plt.gca().invert_yaxis()
plt.show()

A convenient way to see the influence of mixing and photodissociations is to plot both chemical composition on the same plot. For instance:

In [ ]:
#plot of disequibrium and equilibrium compositions

##uncomment these lines if you did not plot abundances before
#activechemistry = tm_diseq.chemistry.activeGasMixProfile
#inactivechemistry = tm_diseq.chemistry.inactiveGasMixProfile
#activechemistry_eq = tmeq.chemistry.activeGasMixProfile
#inactivechemistry_eq = tmeq.chemistry.inactiveGasMixProfile
#pressure_eq = tmeq.pressure.profile
#temperature_eq = tmeq.temperature.profile

colors=['blue','red','green','cyan','pink','orange','gold','olive','magenta','yellow','navy','grey']

fig, ax = plt.subplots()

## iterate over the active gases
for index, gas in enumerate(tm_eq.chemistry.activeGases):
    ax.plot(activechemistry_eq[index, :], pressure_eq,'--', alpha=0.3,color=colors[index])
for index, gas in enumerate(tm_diseq.chemistry.activeGases):   
    ax.plot(activechemistry[index, :], pressure,'-', label = gas,color=colors[index])
    
## iterate over the inactive gases. 
for index, gas in enumerate(tm_eq.chemistry.inactiveGases):
    if gas in ['H2', 'He', 'H']:
        ax.plot(inactivechemistry_eq[index, :], pressure_eq,'--', alpha=0.3)
for index, gas in enumerate(tm_diseq.chemistry.inactiveGases):
    if gas in ['H2', 'He', 'H']:
        ax.plot(inactivechemistry[index, :], pressure,'-', label = gas)
        
    
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e-12, 1)
ax.set_ylim(1e5, 1e-4)
ax.set_xlabel('Volume Mixing Ratio')
ax.set_ylabel('Pressure (mbar)')

#plt.gca().invert_yaxis()
plt.show()

You can see the influence of vertical mixing, with quenching of CH$_4$, NH$_3$, and HCN. You also see the influence of photodissociations, with production of H and HCN in the upper atmosphere, as well as the destruction of NH$_3$.

### Synthetic spectra

#### Transmission spectrum

Let's now plot the corresponding transmission spectrum, binned down to a resolution of R=80.

In [ ]:
native_grid_diseq, rprs_diseq, tau_diseq, _ = res_tm_diseq
### conversion from wavenumber to wavelengths.
native_grid_wl_diseq = 10000/native_grid_diseq

In [ ]:
#from taurex.binning import FluxBinner
#from taurex.util.util import create_grid_res
# We choose the resolution and the range of wavelength
lowres_grid = create_grid_res(80, 0.5, 8.0)
# We initialise the binner with this grid, passing the wavelength and the bin size.
fb = FluxBinner(lowres_grid[:,0], lowres_grid[:,1])
# We can now bin the native spectrum (Note that the grid must be sorted by increasing wavelengths)
lowres_spectrum_diseq = fb.bindown(native_grid_wl_diseq[::-1], rprs_diseq[::-1])

In [ ]:
plt.figure()
plt.plot(native_grid_wl_diseq, rprs_diseq, color='blue', label='High resolution spectrum')
plt.plot(lowres_spectrum_diseq[0], lowres_spectrum_diseq[1], color='orange', label='Low resolution spectrum')
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Transit depth (%)')
plt.legend()
plt.xlim(0.5,8.0)
plt.show()

Let's plot equilibrium and disequilibrium spectra on the same plot:

In [ ]:
plt.figure()
plt.plot(lowres_spectrum_diseq[0], lowres_spectrum_diseq[1], color='blue', label='disequilibrium')
plt.plot(lowres_spectrum_eq[0], lowres_spectrum_eq[1], color='red', label='equilibrium')
plt.legend()
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Transit depth (%)')
plt.xlim(0.5, 8)
plt.show()

You see that the difference between equilibrium and disequilibrium composition between 3 and 4 $\mu$m, around 4.5 $\mu$m and between 6 and 8 $\mu$m.

#### Emission spectrum

Let's plot the corresponding emission spectrum, binned dow to a low resolution:

In [ ]:
# Emission spectrum for disequilibrim composition:
native_grid_em_diseq, fpfs_em_diseq, tau_em_diseq, _ = res_em_diseq
native_grid_wl_em_diseq = 10000/native_grid_em_diseq

In [ ]:
#from taurex.binning import FluxBinner
#from taurex.util.util import create_grid_res
# We choose the resolution and the range of wavelength
lowres_grid = create_grid_res(80, 0.5, 8.0)
# We initialise the binner with this grid, passing the wavelength and the bin size.
fb = FluxBinner(lowres_grid[:,0], lowres_grid[:,1])
# We can now bin the native spectrum (Note that the grid must be sorted by increasing wavelengths)
lowres_spectrum_em_diseq = fb.bindown(native_grid_wl_em_diseq[::-1], fpfs_em_diseq[::-1])

In [ ]:
plt.figure()
plt.plot(native_grid_wl_em_diseq, fpfs_em_diseq, color='blue', label='High resolution spectrum')
plt.plot(lowres_spectrum_em_diseq[0], lowres_spectrum_em_diseq[1], color='orange', label='Low resolution spectrum')
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Transit depth (%)')
plt.legend()
plt.xlim(0.5,8.0)
plt.ylim(0,0.0015)
plt.show()

Let's plot also equilibrium and disequilibrium spectra on the same plot:

In [ ]:
plt.figure()
plt.plot(lowres_spectrum_em_diseq[0], lowres_spectrum_em_diseq[1], color='blue', label='disequilibrium')
plt.plot(lowres_spectrum_em_eq[0], lowres_spectrum_em_eq[1], color='red', label='equilibrium')
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('F$_p$/F$_s$')
plt.xlim(0.5, 8)
plt.ylim(0,0.0015)
plt.legend()
plt.show()

The difference between equilibrium and disequilibrium composition is highly visible on emission.

# Additional informations and plots

In this part, you will see some useful command lines to save results for instance in text files and also some additional plots.

## Thermal profile

Various informations are accessible when the model ran, like temperature, pressure.
For the transmission model at equilibrium (`tm_eq`), the temperature object is accessible with `tm_eq.temperature` (and `em_eq.temperature`). Similarly, the pressure object, is accessible via `tmeq.pressure`(and `em_eq.pressure`). The units are originally in Kelvin and Pascal, for respectively the temperature and the pressure. You can see the data contained in these two objects by writing :


In [ ]:
# Temperature profile
print(tm_eq.temperature.profile)

In [ ]:
# Pressure profile
print(tm_eq.pressure.profile)

And you can of course plot these data. You can choose the unit you prefer to have in your plot. Here, we convert the pressure in mbar instead of Pascal.

In [ ]:
#plot the temperature profile
pressure = tm_eq.pressure.profile*1e-2
temperature = tm_eq.temperature.profile

fig, ax = plt.subplots()

ax.plot(temperature, pressure,'-', color = 'firebrick', linewidth=3, label = 'Temperature')

ax.legend()
ax.set_yscale('log')
ax.set_ylim(1e5, 1e-4)
ax.set_ylabel('Pressure (mbar)')
ax.set_xlabel('Temperature (K)')
#plt.gca().invert_yaxis()
plt.show()

## Information about chemistry

For the transmission model, with disequilibrium chemistry, you can access the chemistry object with `tm_diseq.chemistry`. For the other models, just replace `tm_diseq` with the correct name. 

You can check which gases are included in the model. You can check which gases have IR cross sections and are considered as active for `TauREx`. These gases will contribute to the calculation of the absorption. The other ones are considered for the calculation of chemical kinetics but are 'inactive' for `TauREx`. You can do that with the following lines:

In [ ]:
## Chemistry object:
print(tm_diseq.chemistry)

# the active gases:
print('active gases:', tm_diseq.chemistry.activeGases)
# the inactive:
print('inactive gases:', tm_diseq.chemistry.inactiveGases)


The abundances for the active species can be found by `tm_diseq.chemistry.activeGasMixProfil`

You can also check the elemental abundances used in the model with:


In [ ]:
print('H=',tm_diseq.chemistry.H_abund_dex)
print('C=',tm_diseq.chemistry.C_abund_dex)
print('O=',tm_diseq.chemistry.O_abund_dex)
print('N=',tm_diseq.chemistry.N_abund_dex)

Many other attribute exists, see taurex library at: https://taurex3-public.readthedocs.io/en/latest/api/modules.html

## Save the results

It can be useful to save the results of the model in text files in order to use it afterward.

### Thermal profile

To save the thermal profile you can do: 

In [ ]:
pressure = tm_diseq.pressure.profile
temperature = tm_diseq.temperature.profile

np.savetxt('Outputs/my_temperature_profile.txt', np.array([pressure, temperature]).T)

### Abundances

To save the abundances of all molecules in a single text file, a routine already exists. The output will be a textfile with 2 lines of header giving the species on the first line and the molecular masses on the second line.
The next lines will be the abundances of species at all pressure levels. The columns will be : altitude (km), pressure (mbar), and mixing ratios.
It is convenient to indicate the `name` of the planet, the elemental ratios, and the value of vertical mixing in the filename.

In [ ]:
#filename for disequilibrium abundances
from pychegp.tools import write_to_olivia_format

O_abund_dex = tm_diseq.chemistry.O_abund_dex
C_abund_dex = tm_diseq.chemistry.C_abund_dex
N_abund_dex = tm_diseq.chemistry.N_abund_dex
Kzz_name = 'Kzz_1e'+str(np.log10(tm_diseq.chemistry.Kzz))
species = tm_diseq.chemistry._chegp.species
mole_masses = tm_diseq.chemistry._chegp.mole_masses
altitude = tm_diseq.chemistry._chegp.altitude
P = 1e-2*tm_diseq.chemistry._chegp.P
fm = tm_diseq.chemistry._chegp.fm
folder = 'Outputs/'

filename = os.path.join(folder,f'fractions_molaires_{name}_O{O_abund_dex:.3f}_C{C_abund_dex:.3f}_N{N_abund_dex:.3f}{Kzz_name}.dat')

write_to_olivia_format(filename, species, mole_masses, altitude, P, fm)


For thermochemical equilibrium, you can use :

In [ ]:
#filename for equilibrium abundances
from pychegp.tools import write_to_olivia_format

O_abund_dex = tm_eq.chemistry.O_abund_dex
C_abund_dex = tm_eq.chemistry.C_abund_dex
N_abund_dex = tm_eq.chemistry.N_abund_dex
#Kzz_name = 'Kzz_1e'+str(np.log10(tm_diseq.chemistry.Kzz))
species = tm_eq.chemistry._species
mole_masses = tm_eq.chemistry._molar_masses
altitude = tm_eq.altitude_profile
P = 1e-2*tm_eq._pressure_profile.profile
fm = tm_eq.chemistry._mix_profile
folder = 'Outputs/'

filename = os.path.join(folder,f'fractions_molaires_thermo_{name}_O{O_abund_dex:.3f}_C{C_abund_dex:.3f}_N{N_abund_dex:.3f}.dat')

write_to_olivia_format(filename, species, mole_masses, altitude, P, fm)


### Plot abundances profiles with thermal profile

You can also plot the temperature together with the abundances profiles with the following lines:

In [ ]:
activechemistry = tm_diseq.chemistry.activeGasMixProfile
inactivechemistry = tm_diseq.chemistry.inactiveGasMixProfile
pressure = tm_diseq.pressure.profile*1e-2
temperature = tm_diseq.temperature.profile

fig, ax = plt.subplots()

## iterate over the active gases
for index, gas in enumerate(tm_diseq.chemistry.activeGases):
    ax.plot(activechemistry[index, :], pressure,'-', label = gas)
    
## iterate over the inactive gases. 
for index, gas in enumerate(tm_diseq.chemistry.inactiveGases):
    if gas in ['H2', 'He']:
        ax.plot(inactivechemistry[index, :], pressure,'-', label = gas)
    # You can also add all molecules, without labels !
 #   else:
 #       ax.plot(inactivechemistry[index, :], pressure,'--', alpha = 0.3)
    
ax2=ax.twiny()
ax2.plot(temperature, pressure,'-.', color = 'firebrick', linewidth=3, label = 'Temperature')

ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e-15, 1)
ax.set_ylim(1e5, 1e-4)
ax.set_xlabel('Volume Mixing Ratio')
ax.set_ylabel('Pressure (mbar)')

ax2.legend(loc="lower right")
ax2.set_xlabel('Temperature (K)')
#plt.gca().invert_yaxis()
plt.show()

### Make a movie of the evolution over time

Thanks to the script `pychegp.movie` (available here: *URL*), you can make an movie of the evolution of the atmospheric composition with time. You need to save the composition at different time step in a `history` file:



In [ ]:
#save .history for movie
fms = tm_diseq.chemistry._fms
ts = tm_diseq.chemistry._ts
species = tm_diseq.chemistry._chegp.species
T = tm_diseq.chemistry._chegp.T
conv = True

output = fms, ts, conv
results = {'species': species, 'oderesult': output, 'T' : T, 'P': P}
filename = 'test.history'

import pickle

with open(filename,'wb') as f:
    pickle.dump(results,f)

Then, with the command line on a terminal:
```
python -m pychegp.movie -i test.history -o movie.mp4 
```

you will create the movie.
You can also add an atmospheric composition to compare with, for instance the thermochemical equilibrium with :
```
-c file_to_compare.dat
```

Then you can see the movie with the command:
```
open movie.mp4
```


# Model with an external thermal profile

You can import a thermal profile that has been calculated with a GCM for instance. Here we import the profile of HD 209458b and published originally in Moses et al. 2011.

In [ ]:
# HD 209458b - Moses et al. 2011 profile
from taurex.temperature import TemperatureFile
TemperatureFile = TemperatureFile(filename="pychegp/HD209458b/profil_HD209458b.dat", skiprows=0, temp_col=2,
                                  press_col=1, temp_units='K', press_units='mbar',delimiter=None, reverse=False)


Then, you reconstruct your models (equilibrium, disequilibrium) using this profile, and run them.

In [ ]:
#PLANET
from taurex.planet import Planet
# planet_radius in Jupiter radius
# planet_mass in Jupiter mass
planet = Planet(planet_radius=1.34,planet_mass=0.69)
name = 'HD209458b'

In [ ]:
#transmission model with equilibrium chemistry
from taurex.model import TransmissionModel
tmeq = TransmissionModel(planet=planet,
                       temperature_profile=TemperatureFile,
                       chemistry=chemistryACE,
                       star=star,
                       atm_min_pressure=1e-2,
                       atm_max_pressure=1e7,
                       nlayers=100)

In [ ]:
# add absorption contributions
from taurex.contributions import AbsorptionContribution
tmeq.add_contribution(AbsorptionContribution())
# add cia contributions
from taurex.contributions import CIAContribution
tmeq.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))
# add rayleigh contributions
from taurex.contributions import RayleighContribution
tmeq.add_contribution(RayleighContribution())
# add clouds contribution. Clouds pressure is in Pa
from taurex.contributions import SimpleCloudsContribution
tmeq.add_contribution(SimpleCloudsContribution(clouds_pressure=1e4))

In [ ]:
#build the equilibrium model
tmeq.build()

In [ ]:
#run the equilibrium model
res = tmeq.model()

In [ ]:
#plot of thermochemical equilibrium

activechemistry_eq = tmeq.chemistry.activeGasMixProfile
inactivechemistry_eq = tmeq.chemistry.inactiveGasMixProfile
pressure_eq = tmeq.pressure.profile*1e-2

colors=['blue','orange','green','cyan','navy','red','gold','olive','magenta','yellow','pink','grey']

fig, ax = plt.subplots()

## iterate over the active gases
for index, gas in enumerate(tmeq.chemistry.activeGases):
    ax.plot(activechemistry_eq[index, :], pressure_eq,'-',color=colors[index], label=gas)
    
## iterate over the inactive gases. 
for index, gas in enumerate(tmeq.chemistry.inactiveGases):
    if gas in ['H2', 'He']:
        ax.plot(inactivechemistry_eq[index, :], pressure_eq,'-', label=gas)    


ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e-15, 1)
ax.set_ylim(1e5, 1e-4)
ax.set_xlabel('Volume Mixing Ratio')
ax.set_ylabel('Pressure (mbar)')

plt.show()

In [ ]:
#transmission model with disequilibrium chemistry
from taurex.model import TransmissionModel
tmd = TransmissionModel(planet=planet,
                       temperature_profile=TemperatureFile,
                       chemistry=chemistry,
                       star=star,
                       atm_min_pressure=1e-2,
                       atm_max_pressure=1e7,
                       nlayers=100)

In [ ]:
# add absorption contributions
from taurex.contributions import AbsorptionContribution
tmd.add_contribution(AbsorptionContribution())
# add cia contributions
from taurex.contributions import CIAContribution
tmd.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))
# add rayleigh contributions
from taurex.contributions import RayleighContribution
tmd.add_contribution(RayleighContribution())
# add clouds contribution. Clouds pressure is in Pa
from taurex.contributions import SimpleCloudsContribution
tmd.add_contribution(SimpleCloudsContribution(clouds_pressure=1e4))

In [ ]:
#build the disequilibrium model
tmd.build()

In [ ]:
#run the disequilibrium model
resd = tmd.model()

In [ ]:
#plot of disequilibrium composition
activechemistry = tmd.chemistry.activeGasMixProfile
inactivechemistry = tmd.chemistry.inactiveGasMixProfile
pressure = tmd.pressure.profile*1e-2

colors=['blue','orange','green','cyan','navy','red','gold','olive','magenta','yellow','pink','grey']

fig, ax = plt.subplots()

## iterate over the active gases
for index, gas in enumerate(tmd.chemistry.activeGases):
    ax.plot(activechemistry[index, :], pressure,'-', color=colors[index], label = gas)
    
## iterate over the inactive gases. 
for index, gas in enumerate(tmd.chemistry.inactiveGases):
    if gas in ['H2', 'He']:
        ax.plot(inactivechemistry[index, :], pressure,'-', label = gas)

ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e-15, 1)
ax.set_ylim(1e5, 1e-4)
ax.set_xlabel('Volume Mixing Ratio')
ax.set_ylabel('Pressure (mbar)')

#plt.gca().invert_yaxis()
plt.show()

In [ ]:
#plot of disequibrium and equilibrium compositions

##uncomment these lines if you did not plot abundances before
#activechemistry = tmd.chemistry.activeGasMixProfile
#inactivechemistry = tmd.chemistry.inactiveGasMixProfile
#activechemistry_eq = tmeq.chemistry.activeGasMixProfile
#inactivechemistry_eq = tmeq.chemistry.inactiveGasMixProfile
#pressure_eq = tmeq.pressure.profile
#temperature_eq = tmeq.temperature.profile

colors=['blue','red','green','cyan','pink','orange','gold','olive','magenta','yellow','navy','grey']

fig, ax = plt.subplots()

## iterate over the active gases
for index, gas in enumerate(tmeq.chemistry.activeGases):
    ax.plot(activechemistry_eq[index, :], pressure_eq,'--', alpha=0.3,color=colors[index])
for index, gas in enumerate(tmd.chemistry.activeGases):   
    ax.plot(activechemistry[index, :], pressure,'-', label = gas,color=colors[index])
    
## iterate over the inactive gases. 
for index, gas in enumerate(tmeq.chemistry.inactiveGases):
    if gas in ['H2', 'He']:
        ax.plot(inactivechemistry_eq[index, :], pressure_eq,'--', alpha=0.3)
for index, gas in enumerate(tmd.chemistry.inactiveGases):
    if gas in ['H2', 'He']:
        ax.plot(inactivechemistry[index, :], pressure,'-', label = gas)
        
    
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e-15, 1)
ax.set_ylim(1e5, 1e-4)
ax.set_xlabel('Volume Mixing Ratio')
ax.set_ylabel('Pressure (mbar)')

#plt.gca().invert_yaxis()
plt.show()

In [ ]:
#plot the temperature profile
pressure = tmeq.pressure.profile*1e-2
temperature = tmeq.temperature.profile

fig, ax = plt.subplots()

ax.plot(temperature, pressure,'-', color = 'firebrick', linewidth=3, label = 'Temperature')

ax.legend()
ax.set_yscale('log')
ax.set_ylim(1e5, 1e-4)
ax.set_ylabel('Pressure (mbar)')
ax.set_xlabel('Temperature (K)')
#plt.gca().invert_yaxis()
plt.show()

Let's plot the spectra...

In [ ]:
native_grid_d, rprs_d, tau_d, _ = resd
native_grid_eq, rprs_eq, tau_eq, _ = res

### conversion from wavenumber to wavelengths.
native_grid_wl_d = 10000/native_grid_d
native_grid_wl_eq = 10000/native_grid_eq

#from taurex.binning import FluxBinner
#from taurex.util.util import create_grid_res
# We choose the resolution and the range of wavelength

lowres_grid = create_grid_res(80, 0.5, 8.0)
# We initialise the binner with this grid, passing the wavelength and the bin size.
fb = FluxBinner(lowres_grid[:,0], lowres_grid[:,1])

# We can now bin the native spectrum (Note that the grid must be sorted by increasing wavelengths)
lowres_spectrum_d = fb.bindown(native_grid_wl_d[::-1], rprs_d[::-1])
lowres_spectrum_eq = fb.bindown(native_grid_wl_eq[::-1], rprs_eq[::-1])

In [ ]:
plt.figure()
plt.plot(lowres_spectrum_d[0], lowres_spectrum_d[1], color='blue', label='disequilibrium')
plt.plot(lowres_spectrum_eq[0], lowres_spectrum_eq[1], color='red', label='equilibrium')
plt.legend()
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Transit depth (%)')
plt.xlim(0.5, 8)
plt.show()

# Variations of parameters

Now it's time for you to play with the various parameters (thermal profiles, vertical mixing, C/O ratio) to see their effect on the chemical composition and on the synthetic spectra...